In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_data.shape

(7613, 5)

In [5]:
X = train_data.drop('target',axis=1)
y = train_data['target']

In [6]:
X.head()

,id,keyword,location,text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...


In [7]:

train_data.shape

(7613, 5)

In [8]:
train_data.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [9]:
copy = train_data.copy()
copy.reset_index(inplace = True)

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


In [11]:
copy.head(10)

,index,id,keyword,location,text,target
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [12]:
ps = PorterStemmer()
corpus = []
for i in range(0,len(copy)):
    review1 = re.sub('[^a-zA-Z]',' ',copy['text'][i])
    review1 = review1.lower()
    review1 = review1.split()
    
    review1 = [ps.stem(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    corpus.append(review1)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,2))

X = cv.fit_transform(corpus).toarray()
y =  copy['target']

In [14]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=0)
count_df = pd.DataFrame(X_train,columns=cv.get_feature_names())
#print(count_df.head())



In [15]:
#Multinomial NB Clasifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

from sklearn import metrics
import itertools
classifier.fit(X_train,y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test,pred)
print('accuracy: %0.3f' % score)
cm = metrics.confusion_matrix(y_test,pred)
print(cm)

accuracy: 0.792
[[775 111]
 [206 431]]


In [16]:
#predicting for test data
test_data = pd.read_csv('test.csv')

test_data.shape


(3263, 4)

In [17]:

test_data.reset_index(inplace=True)
corpus = []
for i in range(0,len(test_data)):
    review1 = re.sub('[^a-zA-Z]',' ',test_data['text'][i])
    review1 = review1.lower()
    review1 = review1.split()
    
    review1 = [ps.stem(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    corpus.append(review1)

X = cv.fit_transform(corpus).toarray()

pred = classifier.predict(X)

In [18]:
result = pd.DataFrame()

In [19]:
result['id'] = test_data['id']

In [20]:
result.reset_index(drop=True)

,id
0,0
1,2
2,3
3,9
4,11
...,...
3258,10861
3259,10865
3260,10868
3261,10874


In [21]:
result['target'] = pd.DataFrame(pred)


In [22]:
result.head()

,id,target
0,0,0
1,2,1
2,3,0
3,9,1
4,11,0
